# System Call

![](resources/systemcall01.png)

## system call stack frame

![](resources/systemcall03.png)
![](resources/systemcall04.png)

1. Normal C function call stack frame
  
![](resources/systemcall02.png)

2. System call copy all parameters from the registers to kernel stack to form a normal C function call stack frame. 
   
   Then run the real system call function like calling a normal C function.

--------------------------------